In [ ]:
### This script computes the global diffusion coefficients, parallel optimization over temperatures and proteins for faster evaluation

### Inputs: translational Dt, rotational Dr, RDF of each chain (gmx rdf).
### Output: DG diffusion coefficients

import numpy as np
import os
import time
from multiprocessing import Pool
import sys
sys.path.append('source_codes/')  
from helper_functions import *  
from helper_functions_plots import *  
sys.path.append('../ProteomeClass/')  
from proteome import *

#### USER modifications ##############################################################

base       = "../../../Data.Availability/Upload/TRJ_Psychro_sys4/System4/"               # System at 143 g/L
path       = base + 'Folded/' #'Unfolded/'                                               # Path with temperature folder
path_ind   = base + "Files/"                                                             # Path with Files folder
path_TPR   = base + "Folded/280K" #'Unfolded/280K'                                       # Path of a TPR file
rdf_folder = base + "Examples/RDF/Rdf-Psychro-System4/" #'Rdf-Psychro-System4_unF'       # Path with RDF data
visc_array = base + 'Examples/Viscosity/Psychro/fit_BDtDE.npy' #fit_BDtDE_unF.npy        # Array of viscosity parameters
Dt         = np.load(base + 'Examples/DR_DT/DT-Psychro-System4.npy') #_unF               # Translational diff. array
Dr         = np.load(base + 'Examples/DR_DT/DR-Psychro-System4.npy') #_unF               # Rotational diff. array
nr_proc    = 10                                                                          # Number of processes for parallel optimization
boxsize    = 17                                                                          # Box size

#######################################################################################

# Detect organism, system number etc.
path_parts = base.split("/")
org = path_parts[-3].split('_')[1]
sys_num = path_parts[-2].split("System")[-1]
sys = f"System{sys_num}"
org_short = {'Psychro': 'PA', 'Aquifex': 'AA', 'Ecoli': 'EC'}[org]

# Load the temperatures
Temp = np.array(find_temperatures(path, 'K'))
Temp = sorted(Temp)
Temp = np.array(Temp)

# Load the protein ranges and names
residue_ranges, protein_names = read_residue_ranges(path_ind)
residue_ranges = residue_ranges
protein_names = protein_names
protein_names = rename_duplicates(protein_names)

def optimize_protein(parameters):
    
    p, t, Temp, Dt, Dr, sys_num = parameters
    initial_guess = Dt[t, p] + 0.01
    q = 2
    rdf_file = rdf_folder + "/System" + sys_num + "-" + str(Temp[t]) + "K-pp.%s.RDF.xvg" % (p + 1)
    rdf = read_rdf_xvg(rdf_file)
    rdf = rdf[:, :]
    rdf[:, 0] = rdf[:, 0] * 10  # nm to Angstrom

    start_time = time.time()
    result = minimize(objective_function, initial_guess, args=(q, Dr[t, p], Dt[t, p], rdf), method='Nelder-Mead')
    end_time = time.time()

    elapsed_time = end_time - start_time
    # print("Optimization time for protein", p, "at temperature", Temp[t], ":", elapsed_time, "seconds")
    # print('Translational ', np.round(Dt[t, p],3), 'Rotational ', np.round(Dr[t, p], 3), 'Global ', np.round(result.x[0], 3))
    
    return p,t, result.x[0]


# Define a function for parallel p
def parallel_optimization(parameters, proc):
    with Pool(processes=proc) as pool:  
        results = pool.map(optimize_protein, parameters)
    p_values, t_values, optimized_values = zip(*results)
    return p_values, t_values, optimized_values


# Correct the translational diffusion coefficents
Dt_corr = np.zeros(Dt.shape)
for p in range(Dt.shape[1]):
    Dt_corr[:,p] = Dt[:,p] + get_correction_crowded(boxsize, Temp, path_ind + "protein_indeces.txt",  visc_array)
    
# Correct the rotational diffusion coefficents
Dr_corr = np.zeros(Dr.shape)
for p in range(Dr.shape[1]):
    Dr_corr[:,p] = Dr[:,p] + get_correction_crowded_rot(boxsize, Temp, path_ind + "protein_indeces.txt", visc_array)

# Update correction    
Dr = Dr_corr 
Dt = Dt_corr

# Modify your main loop to create a list of parameters for each protein
parameters_list = []
for (i, p) in enumerate(protein_names):
    for t in range(Dt.shape[0]):
        parameters_list.append((i, t, Temp, Dt, Dr, sys_num))

# Call the parallel function
p_values, t_values, optimized_values = parallel_optimization(parameters_list, nr_proc)
DG = np.zeros((Dt.shape[0], Dt.shape[1]))

# Map the optimized values to the correct indices in DG
for p, t, value in zip(p_values, t_values, optimized_values):
    DG[t, p] = value

# print('Result has finished', DG)


### Weight the diffusion coefficients by hydrogen atoms of chains

In [3]:
def get_DG_weighted(Temp, DG, path_TPR, path_indeces):
        
    # Load the trajectory files to find the proteins
    TPR = glob.glob(os.path.join(path_TPR, '*.tpr'))[0].replace('\\', '/')
    indeces_file = path_indeces + 'chain_indeces.txt'
    prot1 = Proteome(TPR, indeces_file)
    proteins, protein_names = prot1.proteins_list, prot1.protein_names

    # Protein Hydrogen Atoms Weighting
    pHs = []
    for p in proteins:
        p_H = p.select_atoms('name H*')
        pHs.append(p_H.n_atoms)
    weighted_DG = []
    weighted_DG_err = []
    for t in range(DG.shape[0]):
        weighted_mean = np.sum(DG[t, :] * pHs) / np.sum(pHs)
        weighted_DG.append(weighted_mean)
        weighted_variance = np.sum(pHs * (DG[t, :] - weighted_mean)**2) / np.sum(pHs)
        weighted_std = np.sqrt(weighted_variance)
        n = len(DG[t, :])
        standard_error = weighted_std / np.sqrt(n)
        weighted_DG_err.append(standard_error)
    weighted_DG = np.array(weighted_DG)
    weighted_DG_err = np.array(weighted_DG_err)

    return weighted_DG, weighted_DG_err

In [2]:
DG_weighted, DG_weighted_err = get_DG_weighted(Temp, DG, path_TPR, path_ind)

# fig, ax = plt.subplots(1, figsize=(1.3, 2.25), dpi = 300)

# ax.errorbar(Temp, DG_weighted, DG_weighted_err, fmt = 'o', color = 'navy')
# popt, pcov = curve_fit(linear, Temp[:], DG_weighted, sigma=DG_weighted_err, absolute_sigma=True)
# ax.plot(Temp[:], linear(Temp[:], *popt), color = 'navy')
# ax.set_xlabel('$T$ (K)')
# ax.set_ylabel('$D_G$ ($\\mathrm{\\AA}^2$/ns)')

# ax.set_ylim((0,13.5))
# ax.set_xticks([280, 300, 320, 340])
# fig.subplots_adjust(left=0.08, right=0.97, bottom = 0.25, top = 0.9, wspace = 0.3)
